## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [0]:
#This displays the infos of the kernel to which this notebook is connected . id, ...
#https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction
%connect_info
# So an active notebook is exactly one inteactive session with an Ipython kernel, which will execute code sent by the user
# and communicate back results.
# To read more about kernels https://ipython.org/ipython-doc/3/overview.html#ipythonzmq
# To connect to same kernel ipython qtconsole --existing 1ddeb292

{
  "shell_port": 41617,
  "iopub_port": 49607,
  "stdin_port": 60221,
  "control_port": 53091,
  "hb_port": 45017,
  "ip": "127.0.0.1",
  "key": "",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-78bba7ad-839c-46c6-b500-f7a7b7d49106.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [0]:
import numpy as np
import os
from random import shuffle
import re

In [0]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [0]:
import urllib.request
import zipfile
import lxml.etree

In [0]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [0]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [0]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [0]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [0]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [0]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [0]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [0]:
print(len(sentences_ted))
len(sentences_ted[0])

266694


20

In [0]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [0]:
from collections import Counter
# c =Counter(x for xs in sentences_ted for x in xs)
# counts_top_1000 = [x[0] for x in c.most_common(1000)]

c1 = Counter()
for xs in sentences_ted:
    for x in xs:
        c1[x] +=1

        
comm = c1.most_common(1000) #return a list with most commmon words with their countings in a tuple
counts_ted_top1000 = [x[1] for x in comm]
words_top_ted = [x[0] for x in comm]
print(comm[:40])

[('the', 207748), ('and', 149305), ('to', 125169), ('of', 114818), ('a', 105399), ('that', 95146), ('i', 83180), ('in', 78070), ('it', 74738), ('you', 70923), ('we', 67629), ('is', 63251), ('s', 57156), ('this', 49241), ('so', 37014), ('they', 33102), ('was', 30806), ('for', 29713), ('are', 27995), ('have', 27344), ('but', 26732), ('what', 26519), ('on', 25962), ('with', 24706), ('can', 23377), ('t', 22757), ('about', 21246), ('there', 21041), ('be', 20201), ('as', 19488), ('at', 19216), ('all', 19021), ('not', 18626), ('do', 17928), ('my', 17908), ('one', 17551), ('re', 17012), ('people', 16723), ('like', 16273), ('if', 15868)]


Plot distribution of top-1000 words

In [162]:
output_notebook()
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

Loading BokehJS ...

### Part 3: Train Word2Vec

In [0]:
from gensim.models import Word2Vec

In [0]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=1, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [0]:
model_ted.wv.most_similar("man")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.8487130403518677),
 ('guy', 0.8167091012001038),
 ('lady', 0.7809466123580933),
 ('girl', 0.7721682786941528),
 ('soldier', 0.7537262439727783),
 ('boy', 0.742577314376831),
 ('gentleman', 0.7153583765029907),
 ('kid', 0.7065638303756714),
 ('friend', 0.6727961301803589),
 ('poet', 0.6702505946159363)]

In [0]:
model_ted.wv.most_similar("computer")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('machine', 0.7513668537139893),
 ('software', 0.7434959411621094),
 ('robot', 0.6930254697799683),
 ('3d', 0.6911846399307251),
 ('program', 0.6888371706008911),
 ('device', 0.6695933938026428),
 ('interface', 0.6530139446258545),
 ('printer', 0.652542769908905),
 ('simulation', 0.6381321549415588),
 ('mechanical', 0.622725248336792)]

In [0]:
model_ted.wv.most_similar("paris")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('beijing', 0.8725643157958984),
 ('london', 0.8718177080154419),
 ('spain', 0.8575149178504944),
 ('philadelphia', 0.8544884324073792),
 ('2004', 0.8507333993911743),
 ('chicago', 0.850620687007904),
 ('1990', 0.8391937017440796),
 ('mexico', 0.8379186987876892),
 ('boston', 0.8367838859558105),
 ('italy', 0.8366172909736633)]

In [0]:


model_ted.wv.most_similar("telecom")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('somali', 0.9131984710693359),
 ('observatory', 0.9021382927894592),
 ('refinery', 0.8939228653907776),
 ('kruger', 0.8896456956863403),
 ('perez', 0.8845803737640381),
 ('dictator', 0.8821113705635071),
 ('tipped', 0.8810211420059204),
 ('augugliaro', 0.8804455995559692),
 ('subclinical', 0.8794452548027039),
 ('manger', 0.8793189525604248)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [0]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted.wv[words_top_ted]

In [0]:


from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [0]:
output_notebook()
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

We can notice how time words are forming a cluster and they are very close with nearly the same difference between the single and plural ones so
word_vector(weeks) - word_vector(week) == word_vector(minutes) - word_vector(minute) 

<img src="tsne.png">

In [0]:
from IPython.display import Image
Image(filename='tsne1.png')

![Texte alternatif…](https://i.ibb.co/mBqk9q5/tsne.png)

We can calculate the difference between the vectors and conpare it.

![Tsne](https://i.ibb.co/TmHcc0X/tsne1.png)

---



We can notice also that the verbs are forming a cluster so they are close to each other so when if we want to do some text generation it may change a verb by another and still have nearly the same distance with a correct form.

![verbs cluster](verbs_cluster.png)

![Texte alternatif…](https://i.ibb.co/C7S1phk/verbs-cluster.png)

In [0]:
model_ted.wv.wmdistance('minute', 'minutes')

2.5969762104440433

In [0]:
wv1 = model_ted.wv['minute']
wv2 = model_ted.wv['minutes']
cos_dist = np.dot(wv1, wv2) / (np.linalg.norm(wv1) * np.linalg.norm(wv2))
print(cos_dist)

0.7574823


We can notice that word2vec is not using cosine distance between the two word vectors but instead it is using Word Mover’s Distance : **WMD is designed to overcome synonym problem** since it uses word embedding. After that it is using normalized bag-of-words (nBOW) to represent the weight or importance. It assumes that higher frequency implies that it is more important. 

**Paper of the WMD for more details http://proceedings.mlr.press/v37/kusnerb15.pdf**

or this [article](https://towardsdatascience.com/word-distance-between-word-embeddings-cc3e9cf1d632)

### Part 5: Wiki Learnt Representations

Download dataset

In [0]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [0]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [0]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower()).split()
del input_text

In [0]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [0]:
print(sentences_wiki[0])
print(sentences_wiki[1])

['according', 'to', 'tamil', 'versions', 'of', 'the', 'mahabharata', 'the', 'god', 'krishna', 'an', 'incarnation', 'of', 'vishnu', 'also', 'took', 'the', 'form', 'of', 'mohini', 'and', 'married', 'aravan']
['as', 'each', 'van', 'carried', 'coffins', 'and', 'it', 'is', 'known', 'that', 'some', 'funeral', 'trains', 'in', 'the', 'late', 'th', 'century', 'carried', 'over', 'coffins', 'at', 'least', 'six', 'hearse', 'vans', 'must', 'have', 'been', 'in', 'use']


In [0]:
wiki = Counter()
for li in sentences_wiki:
  for word in li:
    wiki[word] +=1

comm_wiki = wiki.most_common(1000) #return a list with most commmon words with their countings in a tuple
counts_wiki_top1000 = [x[1] for x in comm_wiki]
words_top_wiki = [x[0] for x in comm_wiki]
print(comm_wiki[:40])

[('the', 1275152), ('of', 541969), ('and', 493757), ('in', 428100), ('to', 394274), ('a', 354593), ('was', 214003), ('s', 162106), ('on', 152821), ('as', 141588), ('for', 138126), ('that', 137889), ('with', 128177), ('by', 124749), ('is', 102511), ('his', 92504), ('he', 91191), ('at', 90500), ('it', 88537), ('from', 87735), ('were', 70530), ('an', 64060), ('had', 58102), ('which', 54011), ('be', 47489), ('this', 43265), ('but', 42134), ('are', 40722), ('first', 40288), ('not', 39295), ('their', 38565), ('after', 38192), ('one', 37637), ('her', 37261), ('also', 36747), ('its', 36592), ('they', 34654), ('two', 33666), ('have', 32510), ('or', 31629)]


In [0]:
model_wiki = Word2Vec(sentences_wiki, size=100, window=5, min_count=1, workers=4)

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [0]:
model_wiki.wv.most_similar("man")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.7517253160476685),
 ('boy', 0.6900894045829773),
 ('soldier', 0.6575601100921631),
 ('girl', 0.630258321762085),
 ('person', 0.6248133182525635),
 ('dog', 0.601852297782898),
 ('someone', 0.5858423113822937),
 ('men', 0.5660068988800049),
 ('creature', 0.5549851655960083),
 ('mask', 0.5444527864456177)]

In [0]:
model_wiki.wv.most_similar("computer")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('software', 0.7785661220550537),
 ('computers', 0.7559411525726318),
 ('user', 0.7329570055007935),
 ('computing', 0.7256032228469849),
 ('graphics', 0.721127450466156),
 ('handheld', 0.7208528518676758),
 ('hardware', 0.7197294235229492),
 ('desktop', 0.7165095806121826),
 ('controller', 0.7142561674118042),
 ('technology', 0.7024307250976562)]

In [0]:
model_wiki.wv.most_similar("paris")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('berlin', 0.8220511078834534),
 ('vienna', 0.7941826581954956),
 ('copenhagen', 0.7481133937835693),
 ('brussels', 0.743557333946228),
 ('amsterdam', 0.7383338212966919),
 ('stockholm', 0.7249948382377625),
 ('prague', 0.7187910079956055),
 ('moscow', 0.7132781147956848),
 ('helsinki', 0.7088818550109863),
 ('dublin', 0.6930898427963257)]

In [0]:
model_wiki.wv.most_similar("telecom")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('shenzhen', 0.8330349922180176),
 ('abdirahman', 0.8291823863983154),
 ('malheur', 0.827165424823761),
 ('cheongdo', 0.8240376710891724),
 ('hrodna', 0.8199504613876343),
 ('sanchar', 0.8183985352516174),
 ('mylan', 0.8157351016998291),
 ('pvt', 0.8157027959823608),
 ('swingbeat', 0.8152755498886108),
 ('beyle', 0.8132756948471069)]

#### t-SNE visualization

In [0]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki.wv[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [0]:
output_notebook()
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...